We demonstrate here, how parameter estimation on markov logic networks can fit empirical estimations of sufficient statistics.

# Parameter estimation

In [1]:
expressions = {
    "burglary" : ["Burglary"],
    "earthquake" : ["Earthquake"],
    "neighbor" : ["Neighbor"],
    "alarm_works" : ["imp",["or","Burglary","Earthquake"],"Alarm"],
    "neighbor_calls" : ["imp",["and","Alarm",["not","Earthquake"]],"Neighbor"]
}

Situation: We would like to find a probabilistic model, which reproduces the probabilities that the above sentences are satisfied.

In [2]:
targetProbabilities = {
    "burglary" : 0.01,
    "earthquake" : 0.002,
    "neighbor" : 0.1, # 0.01,
    "alarm_works" : 0.999,
    "neighbor_calls" : 0.9995
}

In [3]:
from tnreason import knowledge

estimator = knowledge.EntropyMaximizer(expressionsDict = expressions,
                                       satisfactionDict = targetProbabilities)

In [4]:
optimizedWeights, optimizedFacts = estimator.alternating_optimization()

In [5]:
resultKB = knowledge.HybridKnowledgeBase(
    facts = {key : expressions[key] for key in optimizedFacts},
    weightedFormulas = {key : expressions[key] + [optimizedWeights[key][-1]]
                        for key in optimizedWeights}
)
inferer = knowledge.InferenceProvider(resultKB)

In [6]:
for key in expressions:
  print(key, targetProbabilities[key], inferer.ask(expressions[key]))

burglary 0.01 0.009999987392634122
earthquake 0.002 0.0020000000032835553
neighbor 0.1 0.10000000043386272
alarm_works 0.999 0.9989999999994128
neighbor_calls 0.9995 0.9995


In [7]:
inferer.ask(["Burglary"],evidenceDict={"Alarm":0})

0.0010502759141690682

In [8]:
inferer.ask(["Burglary"],evidenceDict={"Alarm":0, "Neighbor":1})

0.001050275914169068

# From Empirical Samples

In [9]:
samples = inferer.draw_samples(100)

C:\Users\ipobo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tnreason\knowledge\deductive.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampleDf = pd.concat(


In [10]:
empDistribution = knowledge.EmpiricalDistribution(samples)

In [11]:
empInferer = knowledge.InferenceProvider(empDistribution)

In [12]:
empProbabilities = {key : empInferer.ask(expressions[key]) for key in expressions}
print(empProbabilities)

{'burglary': 0.0, 'earthquake': 0.0, 'neighbor': 0.12, 'alarm_works': 1.0, 'neighbor_calls': 1.0}


In [13]:
empProbabilities["burglary"] = 0.01
empProbabilities["earthquake"] = 0.002

In [14]:
estimator = knowledge.EntropyMaximizer(expressionsDict = expressions,
                                       satisfactionDict = empProbabilities)

In [15]:
optimizedWeights, optimizedFacts = estimator.alternating_optimization()

Formula ['imp', ['or', 'Burglary', 'Earthquake'], 'Alarm'] is always satisfied.


C:\Users\ipobo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tnreason\knowledge\weight_estimation.py:68: RuntimeWarning: divide by zero encountered in scalar divide
  return np.log((negValue / posValue) * (empRate / (1 - empRate)))
C:\Users\ipobo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tnreason\knowledge\weight_estimation.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.log((negValue / posValue) * (empRate / (1 - empRate)))


In [16]:
resultKB = knowledge.HybridKnowledgeBase(
    facts = {key : expressions[key] for key in optimizedFacts},
    weightedFormulas = {key : expressions[key] + [optimizedWeights[key][-1]]
                        for key in optimizedWeights}
)
inferer = knowledge.InferenceProvider(resultKB)

In [17]:
print(resultKB)

Hybrid Knowledge Base consistent of
######## probabilistic formulas:
Burglary with weight nan
Earthquake with weight nan
Neighbor with weight nan
(imp_(and_Alarm_(not_Earthquake))_Neighbor) with weight nan
######## logical formulas:
(imp_(or_Burglary_Earthquake)_Alarm)
